# Transformers: Mi történik a pipeline mögött?
Megnézzük, mi történik belül. \
Tokenizer --> Model --> Softmax --> id2label \
A pipeline pár sorral megoldja ezt, de ebben a kódban kibontjuk a lépéseket. \
Egy szövegosztályozós modellel vizsgáljuk meg.

## Előfeldolgozás a Tokenizerrel


*   Letöltjük a szükséges csomagokat
*   Importáljuk az AutoTokenizert
*   Betöltjük a checkpointot (Az eddig tanult súlyokat)
*   Beadjuk a tokenizernek a checkpointokat
*   Megadjuk a tokenizálandó mondatokat
*   Tokenizáljuk a mondatokat. \
A padding=True paraméter azt csinálja, hogy megnézi, melyik mondatnak a legnagyobb a tokenszáma, és az összes többi mondatnak is ilyen hosszúságot ad, hogy a hosszúságok egységesek legyenek. \
Az attention_mask viszont megmondja, hogy mely tokenekre kell figyelni, tehát a rövidebb mondatoknál kinullázza azokat a tokeneket, amik csak kiegészítésként vannak ott. \
A truncation=True a túl hosszú szövegeket levágja a maximális hosszra. \
A return_tensors="pt" megmondja a tokenizernek, hogy milyen formátumban adja vissza az outputot: PyTorch-tensorként, ne sima Python listaként. \
Az output egy dictionary, amelynek kulcsai: "input_ids", "attention_mask" \
Mindkét kulcs értéke egy PyTorch-Tensor.  Az "input_ids" eltárolja a mondatok tokenID-jait, az "attention_mask" eltárolja, hogy mely tokenekre kell figyelni az egyes mondatok esetén.



In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

##Áthaladás a modellen
Egy Sequence Classification modellt építünk.

*   Importáljuk az AutoModelForSequenceClassification osztályt
*   Vesszük a checkpointokat
*   Beadjuk a checkpointot a modellnek
*   A modell megkapja az input_id-kat és az attention_maskokat az **inputs paraméterrel.

Azért kell a **, mert ez szedi szét a dictionary-ben lévő paramétereket és úgy adja át a modellnek. \
Létrejön az outputs objektum, aminek van egy outputs.logits mezője. \
Ez a mező egy 2x2-es mátrix, azért mert 2 mondat x 2 címke. \
Ezért az outputs.logits.shape ezt adja vissza: [2, 2] \
A print(outputs.logits) visszaadja a logits értékeket. \
Ezek még nem valószínűségek. A softmax függvény alakítja át azokká.




In [ ]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs.logits.shape)
print(outputs.logits)

## Postprocessing the output
A softmax függvénnyel átkell alakítani a logitsokat valószínűségekké. \
Ehhez importáljuk a torchot, a softmaxnak beadjuk az outputs.logits értékeket \
A dim=-1 paraméterrel megadjuk, hogy a kapott mátrix melyik tengelye mentén számolja a valószínűségeket. Mivel a mátrix utolsó értéke adja meg a címkék dimenzióját, ezért azt kell használnia a softmaxnak. \
Megkapjuk a valószínűségeket. \
A model.config.id2label megadja, hogy a valószínűségeket tartalmazó tenzor első eleme a "NEGATIVE" címke valószínűsége, a második eleme a "POSITIVE" címke valószínűsége.



In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
model.config.id2label

#A modellek részletesen
A pipeline mögött ilyen modell-objektumok dolgozhatnak:


*   Sequence Classification: AutoModelForSequenceClassification
*   NER: AutoModelForTokenClassification
*   Extractive question Answering: AutoModelForQuestionAnswering
*   Text Generation: AutoModelForCasualLM
*   Translation and Summarization: AutomModelForSeq2SeqLM
*   Alapmodel (BERT-encoder): AutoModel

Ha saját fejet, tanítást, finomhangolást szeretnánk, akkor ezeket kell importálni, nem használjuk a pipeline()-t.





In [ ]:
from transformers import AutoModel
model = AutoModel.from_pretrained("bert-base-cased")
model.save_pretrained("directory_on_my_computer")

# Tokenizerek részletesen
Nézzük meg, mi van a tokenizerek mögött. \
Tokenekre bontást pl. tud végezni a Python split() függvénye. \
Többféle tokenizer van, pl. BertTokenizer, AutoTokenizer. \
Ha az AutoTokenizert használjuk, automatikusan azt a tokenizer osztályt fogja lekérni, ami a checkpointokból kiderül.  

In [ ]:
#Python split function
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

#BertTokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

#AutoTokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


A Tokenizer létrehoz egy dictionaryt, amelynek a kulcsai: input_ids, token_type_ids, attention_mask

In [ ]:
tokenizer("Using a Transformer network is simple")

Így menthetjük el a tokenizert

In [ ]:
tokenizer.save_pretrained("directory_on_my_computer")

A tokenizer függvényt szétbonthatjuk két lépésre:
1. tokenize: ekkor létrehozza a tokeneket
2. convert_tokens_to_ids: ekkor létrehozza a numerikus értékeket (token_id-kat)

DE: használjuk inkább a tokenizer() metódust, mert ebben az esetben nem jön létre az attention_mask (tehát a teljes dictionary), és így hibára fut, amikor a modelnek beadjuk az inputot.

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)
print(tokens)

ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

Dekódolás: amikor a token_id-kat visszalakítjuk szöveggé.

In [ ]:
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

#Minden egybe
Hogyan áll össze a teljs: tokenizer --> modell --> utófeldolgozás \
A tokenizer() metódus képes kezelni több mondatot is, így egy listát adunk neki. \
A tokenizer padding paraméterével megadhatjuk, hogy milyen hosszúak legyenek az egységes tokenhosszúságok. \
Megadhatnánk így is: padding="longest", ekkor a leghosszabb mondatot veszi. Vagy padding="max_length", max_length=8, ekkor 8 lesz az egységes hossz. \
A return_tensors paraméter megadja, hogy milyen formátumba kapjuk meg a kimenetet. ez lehet még "tf" (TensorFlow tensors), "np" (NumPy Arrays), "pt" (PyTorch tensors).

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)
print(output)